# Universidad Autónoma de Yucatán

## Facultad de Matemáticas

### Machine Learning

**Teacher:** Dr. Victor Uc Cetina <[victoruccetina@gmail.com](mailto:victoruccetina@gmail.com)>

**Student:** Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Gradient Descent: Polynomial **Gaussian** Sigmoidal

[GitHub](https://github.com/dayan3847/machine_learning/blob/master/dayan3847/stochastic_gradient_descent_gaussian/gaussian.ipynb)

[Colab](https://colab.research.google.com/github/dayan3847/machine_learning/blob/master/dayan3847/stochastic_gradient_descent_gaussian/gaussian.ipynb)

In [1]:
import numpy as np
import sympy as sp
import plotly as py
from abc import ABC, abstractmethod

In [2]:
points_number: int = 100

In [3]:
def generate_data() -> np.array:
    _x_0: np.array = np.random.rand(points_number)
    _x_1: np.array = np.random.rand(points_number)
    _x_0, _x_1 = np.meshgrid(_x_0, _x_1)
    _e: np.array = np.random.rand(points_number) * .6 - .3
    # generar datos como una función de x_0 y x_1 que sea una función sinusoidal en 3D
    _y: np.array = np.sin(2 * np.pi * _x_0) * np.cos(2 * np.pi * _x_1) + _e
    return np.array([_x_0, _x_0, _y])


my_data: np.array = generate_data()
my_data.shape

(3, 100, 100)

In [5]:
fig = py.graph_objs.Figure()
fig.add_scatter3d(x=my_data[0], y=my_data[1], z=my_data[2], mode='markers')
fig.show()

In [27]:
class Model(ABC):
    def __init__(self, data: np.array):
        self.c: int = 9  # Number of basis functions
        self.w = np.random.rand(self.c) - .5  # Weights
        self.data: np.array = data  # Data
        self.epochs: int = 40  # Number of epochs
        self.a: float = 0.01  # Learning rate

        self.error_history: np.array = np.array([])  # Error history

    @abstractmethod
    def equation_basis_function(self) -> sp.Expr:
        pass

    @abstractmethod
    def basis_function(self, x: float) -> np.array:
        pass

    @abstractmethod
    def equation(self) -> sp.Expr:
        pass

    # Calculate the model value for a simple value
    def hi(self, x: float) -> float:
        bf: np.array = self.basis_function(x)
        return np.dot(self.w, bf)

    # Calculate the model value for a vector
    def h(self, x: np.array) -> np.array:
        return np.array([self.hi(xi) for xi in x])

    def activate(self, x: float) -> float:
        return 1 / (1 + np.exp(-x))
    
    def gi(self, x: float) -> float:
        return self.activate(self.hi(x))
    
    def g(self, x: np.array) -> np.array:
        return np.array([self.gi(xi) for xi in x])
    
    def e(self) -> float:
        return np.sum((self.h(self.data[0]) - self.data[1]) ** 2) / 2

    def summary(self):
        print('Model: {}'.format(self.__class__.__name__))
        print('Error: {}'.format(round(self.e(), 2)))

    def train(self):
        for _ in range(self.epochs):
            self.save()
            self.train_step()

    def train_step(self):
        a = self.a
        x_ = self.data[0]
        y_ = self.data[1]
        for x, y in zip(x_, y_):
            b = self.basis_function(x)
            h = self.hi(x)
            for i in range(self.c):
                self.w[i] -= a * (h - y) * b[i]

    def save(self):
        self.error_history = np.append(self.error_history, self.e())

In [28]:
class ModelPolynomial(Model):
    def __init__(self, d: np.array):
        super().__init__(d)
        self.n: np.array = np.arange(self.c)

    def equation_basis_function(self) -> sp.Expr:
        x: sp.Symbol = sp.Symbol('x')
        n: sp.Symbol = sp.Symbol('n')
        return x ** n

    def equation(self) -> sp.Expr:
        r: sp.Expr = sp.sympify(0)
        for w, n in zip(self.w, self.n):
            w_: float = round(w, 2)
            r += w_ * self.equation_basis_function().subs({'n': n})
        return r

    def basis_function(self, x: float) -> np.array:
        x_: np.array = np.full(self.c, x)
        return x_ ** self.n


model_p: Model = ModelPolynomial(my_data)
model_p.summary()
model_p.equation_basis_function()

Model: ModelPolynomial
Error: 25.8


x**n

In [29]:
model_p.equation()

-0.43*x**8 - 0.15*x**7 - 0.1*x**6 - 0.37*x**5 - 0.43*x**4 + 0.23*x**3 + 0.37*x**2 + 0.02*x - 0.21

In [30]:
class ModelGaussian(Model):
    def __init__(self, d: np.array):
        super().__init__(d)
        self.m: np.array = np.linspace(0, 1, self.c)
        self.s: np.array = np.full(self.c, 0.1)

    def equation_basis_function(self) -> sp.Expr:
        x: sp.Symbol = sp.Symbol('x')
        m: sp.Symbol = sp.Symbol('m')
        s: sp.Symbol = sp.Symbol('s')
        return sp.exp(-1 * ((x - m) ** 2) / (2 * (s ** 2)))

    def equation(self) -> sp.Expr:
        r: sp.Expr = sp.sympify(0)
        for w, m, s in zip(self.w, self.m, self.s):
            w_: float = round(w, 2)
            r += w_ * self.equation_basis_function().subs({'m': m, 's': s})
        return r

    def basis_function(self, x: float) -> np.array:
        x_: np.array = np.full(self.c, x)
        return np.exp(-1 * ((x_ - self.m) ** 2) / (2 * (self.s ** 2)))


model_g: Model = ModelGaussian(my_data)
model_g.summary()
model_g.equation_basis_function()

Model: ModelGaussian
Error: 46.11


exp(-(-m + x)**2/(2*s**2))

In [31]:
model_g.equation()

0.16*exp(-50.0*(x - 0.125)**2) + 0.01*exp(-50.0*(x - 0.25)**2) - 0.2*exp(-50.0*(x - 0.375)**2) + 0.5*exp(-50.0*(x - 0.5)**2) - 0.17*exp(-50.0*(x - 0.625)**2) + 0.48*exp(-50.0*(x - 0.75)**2) + 0.37*exp(-50.0*(x - 0.875)**2) - 0.13*exp(-50.0*(x - 1.0)**2) - 0.34*exp(-50.0*x**2)

In [32]:
class ModelSigmoidal(Model):
    def __init__(self, d: np.array):
        super().__init__(d)
        self.m: np.array = np.linspace(0, 1, self.c)
        self.s: np.array = np.full(self.c, 0.1)

    def equation_basis_function(self) -> sp.Expr:
        x: sp.Symbol = sp.Symbol('x')
        m: sp.Symbol = sp.Symbol('m')
        s: sp.Symbol = sp.Symbol('s')
        return 1 / (1 + sp.exp((m - x) / s))

    def equation(self) -> sp.Expr:
        r: sp.Expr = sp.sympify(0)
        for w, m, s in zip(self.w, self.m, self.s):
            w_: float = round(w, 2)
            r += w_ * self.equation_basis_function().subs({'m': m, 's': s})
        return r

    def basis_function(self, x: float) -> np.array:
        x_: np.array = np.full(self.c, x)
        return 1 / (1 + np.exp((self.m - x_) / self.s))


model_s: Model = ModelSigmoidal(my_data)
model_s.summary()
model_s.equation_basis_function()

Model: ModelSigmoidal
Error: 34.38


1/(exp((m - x)/s) + 1)

In [33]:
model_s.equation()

0.2/(1 + 22026.4657948067*exp(-10.0*x)) + 0.01/(1 + 6310.68810808902*exp(-10.0*x)) - 0.16/(1 + 1808.04241445606*exp(-10.0*x)) - 0.25/(1 + 518.012824668342*exp(-10.0*x)) + 0.08/(1 + 148.413159102577*exp(-10.0*x)) - 0.18/(1 + 42.5210820000628*exp(-10.0*x)) + 0.33/(1 + 12.1824939607035*exp(-10.0*x)) + 0.34/(1 + 3.49034295746184*exp(-10.0*x)) + 0.05/(1 + exp(-10.0*x))

In [34]:
fun['h_pol'] = model_p.h(fun['x'])
fun['h_gau'] = model_g.h(fun['x'])
fun['h_sig'] = model_s.h(fun['x'])

In [35]:
fig = py.graph_objs.Figure()
fig.add_scatter(x=fun['x'], y=fun['sin'], name=str(gen_function), line_dash='dash', line_color='gray')
fig.add_scatter(x=my_data[0], y=my_data[1], name='Points', mode='markers', marker_color='red')
fig.add_scatter(x=fun['x'], y=fun['h_pol'], name='Polynomial', line_color='blue')
fig.add_scatter(x=fun['x'], y=fun['h_gau'], name='Gaussian', line_color='green')
fig.add_scatter(x=fun['x'], y=fun['h_sig'], name='Sigmoidal', line_color='purple')
fig.show()

In [36]:
model_p.summary()
model_p.train()
model_p.summary()
model_p.equation()

Model: ModelPolynomial
Error: 25.8
Model: ModelPolynomial
Error: 8.38


0.45*x**8 + 0.62*x**7 + 0.48*x**6 - 0.04*x**5 - 0.47*x**4 - 0.3*x**3 - 0.68*x**2 - 1.12*x + 0.69

In [37]:
model_g.summary()
model_g.train()
model_g.summary()
model_g.equation()

Model: ModelGaussian
Error: 46.11
Model: ModelGaussian
Error: 1.5


0.5*exp(-50.0*(x - 0.125)**2) + 0.63*exp(-50.0*(x - 0.25)**2) + 0.22*exp(-50.0*(x - 0.375)**2) + 0.33*exp(-50.0*(x - 0.5)**2) - 0.69*exp(-50.0*(x - 0.625)**2) - 0.55*exp(-50.0*(x - 0.75)**2) - 0.34*exp(-50.0*(x - 0.875)**2) - 0.02*exp(-50.0*(x - 1.0)**2) - 0.2*exp(-50.0*x**2)

In [38]:
model_s.summary()
model_s.train()
model_s.summary()
model_s.equation()

Model: ModelSigmoidal
Error: 34.38
Model: ModelSigmoidal
Error: 2.91


0.66/(1 + 22026.4657948067*exp(-10.0*x)) + 0.61/(1 + 6310.68810808902*exp(-10.0*x)) + 0.06/(1 + 1808.04241445606*exp(-10.0*x)) - 0.76/(1 + 518.012824668342*exp(-10.0*x)) - 0.88/(1 + 148.413159102577*exp(-10.0*x)) - 0.97/(1 + 42.5210820000628*exp(-10.0*x)) + 0.12/(1 + 12.1824939607035*exp(-10.0*x)) + 0.66/(1 + 3.49034295746184*exp(-10.0*x)) + 0.56/(1 + exp(-10.0*x))

In [39]:
fig = py.graph_objs.Figure()
fig.add_scatter(y=model_p.error_history, name='Polynomial', line_color='blue')
fig.add_scatter(y=model_g.error_history, name='Gaussian', line_color='green')
fig.add_scatter(y=model_s.error_history, name='Sigmoidal', line_color='purple')
fig.show()

In [40]:
fun['h_pol'] = model_p.h(fun['x'])
fun['h_gau'] = model_g.h(fun['x'])
fun['h_sig'] = model_s.h(fun['x'])
fig = py.graph_objs.Figure()
fig.add_scatter(x=fun['x'], y=fun['sin'], name=str(gen_function), line_dash='dash', line_color='gray')
fig.add_scatter(x=my_data[0], y=my_data[1], name='Points', mode='markers', marker_color='red')
fig.add_scatter(x=fun['x'], y=fun['h_pol'], name='Polynomial', line_color='blue')
fig.add_scatter(x=fun['x'], y=fun['h_gau'], name='Gaussian', line_color='green')
fig.add_scatter(x=fun['x'], y=fun['h_sig'], name='Sigmoidal', line_color='purple')
fig.show()